In [1]:
import arkouda as ak
import arachne as ar
import pandas as pd
import numpy as np
import networkx as nx
import time
import json

    _         _                   _       
   / \   _ __| | _____  _   _  __| | __ _ 
  / _ \ | '__| |/ / _ \| | | |/ _` |/ _` |
 / ___ \| |  |   < (_) | |_| | (_| | (_| |
/_/   \_\_|  |_|\_\___/ \__,_|\__,_|\__,_|
                                          

Client Version: v2024.06.21+0.gcf6eeacde.dirty


In [2]:
# These are just wedges, we do not need subgraph isomorphism for these.
src0 = [    1,     1]
dst0 = [10002, 10003]
connection_type0 = ['n', 'n']
connection_type0 = [0, 0]
color0 = ['b', 'b']

src1 = [    1,     1, 10003, 10003]
dst1 = [10002, 10003, 30004, 30005]
connection_type1 = ['n', 'n', 'n', 'n']
connection_type1 = [0, 0, 0, 0]
color1 = ['b', 'b', 'b', 'b']

src2 = [    1,     1, 40005]
dst2 = [10002, 10003,     1]
connection_type2 = ['n', 'n', 's']
connection_type2 = [0, 0, 1]
color2 = ['b', 'b', 'b_r']

src3 = [    1,     1, 40005, 40005, 50008]
dst3 = [10002, 10003, 50008,     1, 10003]
connection_type3 = ['n', 'n', 'n', 's', 's']
connection_type3 = [0, 0, 0, 1, 1]
color3 = ['b', 'b', 'r', 'b_r', 'b_r']

src4 = [1, 1, 10003, 10003, 60007]
dst4 = [10002, 10003, 30004, 30005, 1]
connection_type4 = ['n', 'n', 'n', 'n', 's']
connection_type4 = [0, 0, 0, 0, 1]
color4 = ['b', 'b', 'b', 'b', 'b_r']

src5 = [1, 1, 10003, 10003, 60007, 60007, 70010]
dst5 = [10002, 10003, 30004, 30005, 70010, 1, 30005]
connection_type5 = ['n', 'n', 'n', 'n', 'n', 's', 's']
connection_type5 = [0, 0, 0, 0, 0, 1, 1]
color5 = ['b', 'b', 'b', 'b', 'r', 'b_r',  'b_r']

src6 = [1, 1, 40005, 40005, 80009, 80009, 10003, 90010]
dst6 = [10002, 10003, 50006, 50007, 90010, 90011, 50006, 50007]
connection_type6 = ['n', 'n', 'n', 'n', 'n', 'n', 's', 's']
connection_type6 = [0, 0, 0, 0, 0, 0, 1, 1]
color6 = ['b', 'b', 'r', 'r', 'y', 'y', 'b_r', 'b_y']

src7 = [1, 10002, 40005, 60007, 80009]
dst7 = [10002, 20003, 1, 10002, 20003]
connection_type7 = ['n', 'n', 's', 's', 's']
connection_type7 = [0, 0, 1, 1, 1]
color7 = ['b', 'b', 'b_r', 'b_y', 'r_y']

subgraphs_as_list_of_tuples = [ 
                                (src0, dst0, connection_type0, color0, "0"),
                                (src3, dst3, connection_type3, color3, "3"),
                                (src2, dst2, connection_type2, color2, "2"),
                                (src7, dst7, connection_type7, color7, "7"),
                                (src5, dst5, connection_type5, color5, "5"),
                                (src4, dst4, connection_type4, color4, "4"),
                                (src1, dst1, connection_type1, color1, "1"),
                                (src6, dst6, connection_type6, color6, "6")
                            ]

In [3]:
df = pd.read_csv("/scratch/users/oaa9/arkouda-njit/arachne/data/OL_dataset.csv")

/tmp/ipykernel_2377603/1960765740.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/scratch/users/oaa9/arkouda-njit/arachne/data/OL_dataset.csv")


In [4]:
# Change to integers.
print(np.unique(df["connection_type"]))
df["connection_type"] = np.where(df["connection_type"] == "n", 0, 1)
print(np.unique(df["connection_type"]))

['n' 's']
[0 1]


In [5]:
# NOTE: Make sure to change the server name to whatever is applicable in your environment. If running locally, then use only ak.connect().
ak.connect("n115", 5555)

connected to arkouda server tcp://*:5555


In [6]:
ak.client.maxTransferBytes *= 2

In [7]:
g = ar.PropGraph()
transformed_dataset = ak.DataFrame(df.to_dict(orient='list'))
g.load_edge_attributes(transformed_dataset, source_column="src", destination_column="dst", 
                            relationship_columns=["s_bef", "s_bef_x", "s_bef_y", "s_bef_z", "s_af", "s_af_x", 
                                                    "s_af_y", "s_af_z", 's_x', "s_y", "s_z", "s_distance", "d_bef",
                                                    "d_bef_x", "d_bef_y", "d_bef_z", "d_af", "d_af_x", "d_af_y", "d_af_z",
                                                    "d_x", "d_y", "d_z", "d_distance", "n_id", "connection_type"])

print(f"Built graph with {len(g)} vertices and {g.size()} edges")

Built graph with 667711 vertices and 962796 edges


In [8]:
def get_mapping(g, subgraph, iso_cap):
    # Process subgraph information for mapping after subgraph isomorphism is invoked.
    src_sub, dst_sub = subgraph.edges()
    src_sub = src_sub.to_ndarray()
    dst_sub = dst_sub.to_ndarray()
    subgraph_nodes = sorted(list(np.unique(np.concatenate((src_sub, dst_sub)))))

    start = time.time()
    if iso_cap > 0 :
        isos = ar.subgraph_isomorphism(g, subgraph, algorithm_type="si", return_isos_as="complete", semantic_check="or", size_limit=iso_cap)
    else:
        isos = ar.subgraph_isomorphism(g, subgraph, algorithm_type="si", return_isos_as="complete", semantic_check="or")
    end = time.time()
    
    # Extract the returned array information from subgraph_isomorphism.
    isos_by_vertices = isos[0]
    isos_by_vertices_map = isos[1]
    isos_by_edges_src = isos[2]
    isos_by_edges_dst = isos[3]

    if len(isos_by_vertices) % len(subgraph) != 0:
        raise ValueError("The length of isomorphisms is not a multiple of the number of subgraph nodes.")

    # Get the number of motifs found.
    number_isos_found = len(isos_by_vertices) // len(subgraph_nodes)
    print(f"Finding {number_isos_found:_} motifs took: {end-start} seconds.")

    # Prepare the returned isomorphisms as a 2D array.
    start = time.time()
    isos_ndarray = isos_by_vertices.to_ndarray()
    hostgraph_nodes = isos_ndarray.reshape(-1, len(subgraph_nodes))
    end = time.time()
    print(f"Reshaping isomorphisms took: {end-start} seconds.")

    # Create all mappings at once using a list comprehension.
    start = time.time()
    all_mappings = [
        {int(k): int(v) for k, v in zip(subgraph_nodes, hostgraph_nodes[i])}
        for i in range(number_isos_found)
    ]
    end = time.time()
    print(f"Generating mappings took: {end-start} seconds.")

    return (all_mappings,(isos_by_edges_src,isos_by_edges_dst))

In [9]:
def motif_to_viss(src, dst, connection_type, iso_cap):    
    subgraph_dict = {
        "src": src,
        "dst": dst,
        "connection_type": connection_type
        }
    
    subgraph = ar.PropGraph()
    df = ak.DataFrame(subgraph_dict)
    subgraph.load_edge_attributes(df, source_column="src", destination_column="dst", 
                                relationship_columns=["connection_type"])
    
    node_mapping = get_mapping(g, subgraph, iso_cap)
    return node_mapping

In [10]:
def get_post_processed_mappings(subgraph_src, subgraph_dst, color, mapping):
    start = time.time()
    dict_to_check_against = {}
    for c in color:
        if c not in dict_to_check_against:
            dict_to_check_against[c] = 1
        else:
            dict_to_check_against[c] += 1

    isos_by_vertices = mapping[0]
    isos_by_edges_src = mapping[1][0]
    isos_by_edges_dst = mapping[1][1]
    
    num_edges_subgraph = len(subgraph_src)
    number_isos_found = len(isos_by_edges_src) // len(subgraph_src)

    indices = ak.find([isos_by_edges_src,isos_by_edges_dst],[g.edge_attributes["src"], g.edge_attributes["dst"]])
    vals = g.edge_attributes["n_id"][indices]

    # TODO: Can the below be done with Arkouda?
    isos_by_edges_src_ndarray = np.split(isos_by_edges_src.to_ndarray(), number_isos_found)
    isos_by_edges_dst_ndarray = np.split(isos_by_edges_dst.to_ndarray(), number_isos_found)
    vals_ndarray = np.split(vals.to_ndarray(), number_isos_found)

    matches = 0
    curr_mapping_id = 0
    final_mappings = []
    for src,dst,vals in zip(isos_by_edges_src_ndarray,isos_by_edges_dst_ndarray,vals_ndarray):
        inner_matches = {}
        for val in vals:
            if val not in inner_matches:
                inner_matches[val] = 1
            else:
                inner_matches[val] += 1
        if sorted(inner_matches.values()) == sorted(dict_to_check_against.values()):
            final_mappings.append(isos_by_vertices[curr_mapping_id])
            matches += 1
        curr_mapping_id += 1

    end = time.time()
    print(f"Before post processing number of motifs found was {number_isos_found} and after was {matches}")
    print(f"Post processing took: {end-start} seconds.")

    return final_mappings

In [ ]:
all_mappings = {}
cap = 0
send_to_file = True
for tup in subgraphs_as_list_of_tuples:
    subgraph_id = int(tup[4])
    subgraph_name = f"subgraph{subgraph_id}"
    subgraph_src = tup[0]
    subgraph_dst = tup[1]
    subgraph_connection_type = tup[2]
    subgraph_color = tup[3]

    print(f"Querying for {subgraph_name}...")
    curr_mapping = motif_to_viss(subgraph_src, subgraph_dst, subgraph_connection_type, cap)
    all_mappings[subgraph_id] = curr_mapping

    print(f"Post processing {subgraph_name}...")
    map_to_write = get_post_processed_mappings(subgraph_src, subgraph_dst, subgraph_color, all_mappings[subgraph_id])

    if send_to_file:
        filename = subgraph_name + f"_motifs_capped_at_{cap}.json" if cap != 0 else subgraph_name + f"_motifs.json"
        with open(filename, "w") as f:
            json.dump(map_to_write, f)
    
    print()

Querying for subgraph0...
Finding 696_460 motifs took: 2.4765231609344482 seconds.
Reshaping isomorphisms took: 0.18685626983642578 seconds.
Generating mappings took: 0.6457834243774414 seconds.
Post processing subgraph0...
Before post processing number of motifs found was 696460 and after was 696460
Post processing took: 5.734685182571411 seconds.

Querying for subgraph3...
Finding 5_048 motifs took: 2.877610445022583 seconds.
Reshaping isomorphisms took: 0.012549400329589844 seconds.
Generating mappings took: 0.0061342716217041016 seconds.
Post processing subgraph3...
Before post processing number of motifs found was 5048 and after was 5048
Post processing took: 2.4994661808013916 seconds.

Querying for subgraph2...
Finding 191_690 motifs took: 3.6195456981658936 seconds.
Reshaping isomorphisms took: 0.07360506057739258 seconds.
Generating mappings took: 0.2176060676574707 seconds.
Post processing subgraph2...
Before post processing number of motifs found was 191690 and after was 191

In [ ]:
# Convert graph to NetworkX and run their subgraph isomorphism.
G = nx.from_pandas_edgelist(df, source="src", target="dst", edge_attr=True, create_using=nx.DiGraph())

In [ ]:
def edge_match(e1, e2):
    return e1["connection_type"] == e2["connection_type"]

for tup in subgraphs_as_list_of_tuples:
    h = nx.DiGraph()
    src, dst, connection_type, color, identifier = tup
    for i in range(len(src)):
        u = src[i]
        v = dst[i]
        h.add_edge(u, v, connection_type=connection_type[i], color=color[i])

    print(f"Querying for {identifier}...")
    start = time.time()
    isomatcher = nx.isomorphism.DiGraphMatcher(G,h,edge_match=edge_match)
    motifs = list(isomatcher.subgraph_monomorphisms_iter())
    end = time.time()
    print(f"Finding {len(motifs):_} motifs took: {end-start} seconds.")